# Selección de cripto activos para una cartera de inversión

![picture](https://drive.google.com/uc?export=view&id=1ihM3JEKP-XPaoX9b5bwymu_IF4ntTlz-)

# Trabajo Práctico 3 -  Aprendizaje Supervisado

# Introducción

Una tarea fundamental en el modelado financiero es la predicción del comportamiento de los precios en un futuro cercano. Usando la serie de precios histórica de los activos como datos de entrenamiento, deseamos predecir si el precio subirá o no, y en qué medida, es decir, nos interesa también el rendimiento.

La predicción de retornos de criptoactivos es un gran desafío para la comunidad de aprendizaje automático debido a la alta volatilidad de los activos, la manipulación del mercado y la naturaleza no estacionaria de los datos, entre otras cuestiones.  

En este práctico aplicaremos algunas técnicas de aprendizaje automático supervisado para predecir el movimiento de activos, a partir de los features que hemos trabajado en el práctico anterior. Exploraremos modelos simples de regresión y clasificación. Para validar los modelos procedemos a evaluar su bondad de ajuste, es decir, que "tan bueno" es el poder predictivo del modelo y comparar su desempeño.

📌 **Conjunto de Entrenamiento y Test**

Para entrenar modelos de ML y seleccionar el más adecuado, debemos particionar los datos en conjuntos de entrenamiento y test (o entrenamiento, validación y test). Si bien la práctica más habitual es seleccionar estos conjuntos al azar, esta metodología no es conveniente cuando trabajamos con series de tiempo, ya que romper la estructura de los datos significa romper la autocorrelación, que es lo que da sentido al análisis. Frente a esto, una posibilidad es realizar la división de datos a partir del procedimiento [Walk-Forward Validation](https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/), otra es implementar lo que se conoce como [timeSeriesSplit](https://scikit-learn.org/stable/modules/cross_validation.html#time-series-split).

¡Los invito a indagar sobre estas metodologías!


📌 **Variables Predictoras**

Los predictores son datos que consideramos relevantes para el comportamiento del mercado. En nuestro caso, trabajaremos con los indicadores técnicos calculados en el trabajo práctico anterior y algunos que sumaremos al análisis, pero estos pueden ser muy diversos, como datos de sentimiento, datos de amplitud, datos básicos, gubernamentales, etc., que nos ayudarán a hacer nuestras predicciones. 



📌 **Modelo Baseline**

Establecer un baseline es esencial para cualquier problema de predicción. Este modelo nos brinda un punto de comparación, nos da una idea de cuan bien otros modelos rindieron en el problema en cuestión.

Si la performance de un modelo no alcanza o es similiar a la del baseline, entonces debemos seguir trabajando para mejorar el modelo u optar por algún otro. Nuestro baseline debe ser fácil de implementar y naive. 


Algunas opciones simples para baseline: [link](https://medium.com/analytics-vidhya/benchmarking-methods-for-deep-learning-based-time-series-forecast-ec45f78b61e2).

# 📖 ¡Actividades!

En este práctico evaluaremos el potencial de modelos predictivos simples sobre algunos activos. Esta será la base para construir una canasta de activos con riesgo controlado. 

**Objetivo: predecir el movimiento del precio de cada activo para los próximos n días (tomaremos n=1, n=7)**

## Parte 0 - Feature Engineer

Los invito a explorar y agregar otras características además de las trabajadas anteriormente. Como en el práctico pasado, pueden usar la librería talib o calcularlos ustedes.

- [Indicadores técnicos](https://coinmarketcap.com/alexandria/article/technical-analysis-101-the-best-technical-indicators-for-crypto-and-stocks)
- [Indicadores técnicos en python](https://towardsdatascience.com/building-a-comprehensive-set-of-technical-indicators-in-python-for-quantitative-trading-8d98751b5fb)

## Parte I - Clasificación

Por ahora nos concentraremos en predecir si el precio del activo subirá o bajará (es decir, si el retorno a n días será positivo o negativo). Para ello entrenaremos algunos modelos de clasificación.

- DecisionTreeClassifier
- RandomForestClassifier
- XGBClassifier

En este caso, podemos definir nuestro target como 1 o 0:

- Si el rendimiento a n días fue positivo el target tomará el valor 1.
- Si el rendimiento a n días fue negativo o cero el target tomará el valor 0.

Utilicen métricas adecuadas para la evaluación de los modelos (accuracy, recall, precision, etc) 


## Parte II - Regresión

Ahora nuestro objetivo es implementar modelos sobre los activos para predecir su tasa de cambio (ya sea respecto al precio open o close). Consideren los modelos de regresión:

- Desicion tree
- Randon Forest
- XGBoost


Evalúen los resultados, con algunas métricas útiles, tales como Error Cuadrático Medio (RMSE) y el Error Absoluto Medio (MAE). ¿Qué pueden decir?


### ¡Para tener en cuenta!

- Para los modelos de regresión y clasificación elijan un modelo baseline.
- Definir periodos de entrenamiento y test: utilizar librerías adecuadas para series de tiempo.
- Dado el periodo de entrenamiento, chequeen si los datos están balaceados.  
- Analicen la necesidad de estandarizar los datos, según los modelos empleados.
- Ajuste de hiperparámetros de los modelos: dado que los parámetros pueden tomar valores en un amplio rango es recomendable uilizar GridSearchCV. 



## Opcional - Predicción usando Long short-term memory (LSTM)

Las redes de este tipo son adecuadas para clasificar, procesar y hacer predicciones basadas en datos de series temporales, ya que pueden almacenar información pasada. Esto es importante en nuestro caso porque el precio anterior de una acción es crucial para predecir su precio futuro.

Comparen la performance de los modelos anteriores (regresión y/o clasificación) con una red neuronal recurrente (LSTM). 

# Fecha de entrega

- __Versión preliminar: 29/7__ 

# Condiciones de entrega

Realizar un informe en el cual se presenten los resultados y conclusiones del análisis desarrollado. El mismo debe estar pensado para un público técnico pero que desconoce los aspectos propios del problema a resolver. Dicho informe puede ser un notebook, a condición de que en el mismo se efectúe un análisis escrito suficientemente detallado de los resultados. 

In [1]:
import pandas as pd
import numpy as np

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from scipy.stats import norm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from pylab import rcParams
import statsmodels.api as sm

from sklearn.preprocessing import MinMaxScaler

from pandas import Series, DataFrame

import seaborn as sns
import os
import time
import missingno as msno

import talib as ta
import datetime as dt

import warnings
warnings.filterwarnings("ignore") 

In [2]:
df = pd.read_csv('../2tp_analisisYCuracion/Activos limpios con features - 2tp.csv')

In [3]:
df.head()

,open,high,low,close,volume,active,day,month,year,weekday,profit,real_volume,EMA200,EMA100,EMA50,EMA20,RSI200,ADX200,MACD
0,266.1,275.0,262.7,272.3,37025.276,AAVEUSDT,3,1,2022,Monday,2.329951,1.008198e+07,235.641863,242.201303,255.302584,260.680336,51.444866,8.250582,2.076257
1,312.1,316.9,310.2,312.4,15282.570,AAVEUSDT,25,10,2021,Monday,0.096123,4.774275e+06,310.654344,307.708981,310.507227,312.431351,49.611024,7.611853,-0.394056
2,312.4,312.5,308.9,311.6,7623.416,AAVEUSDT,25,10,2021,Monday,-0.256082,2.375456e+06,310.663753,307.786031,310.550081,312.352175,49.570851,7.598432,-0.391331
3,311.6,322.5,311.1,322.1,18258.332,AAVEUSDT,25,10,2021,Monday,3.369705,5.881009e+06,310.777547,308.069476,311.003019,313.280539,50.103818,7.579242,0.452871
4,321.9,331.7,319.6,330.2,34300.271,AAVEUSDT,26,10,2021,Tuesday,2.578441,1.132595e+07,310.970805,308.507704,311.755841,314.891916,50.509340,7.554866,1.755275


In [4]:
df.drop(columns=[ 'EMA200', 'EMA100', 'EMA50',
       'EMA20', 'RSI200', 'ADX200', 'MACD'], inplace=True)

## Parte 0 - Feature Engineer

### Funciones

In [5]:
def _extract_close(data, active):
    values = None

    if isinstance(data, Series):
        values = data.values
    else:
        if "close" in data.columns:
            values = data[data['active'] == active]['close']

    if values is None:
        raise ValueError(
            "data must be Pandas Series or DataFrame with a 'last' or 'close' column")

    return values

In [6]:
def _extract_high(data, active):
    values = None

    if isinstance(data, Series):
        values = data.values
    else:
        if "high" in data.columns:
            values = data[data['active'] == active]['high']

    if values is None:
        raise ValueError(
            "data must be Pandas Series or DataFrame with a 'last' or 'close' column")

    return values

In [7]:
def _extract_low(data, active):
    values = None

    if isinstance(data, Series):
        values = data.values
    else:
        if "low" in data.columns:
            values = data[data['active'] == active]['low']

    if values is None:
        raise ValueError(
            "data must be Pandas Series or DataFrame with a 'last' or 'close' column")

    return values

In [8]:
def STOCHRSI(data, active):
    closing_prices = _extract_close(data, active)
    return ta.STOCHRSI(closing_prices) 

In [9]:
def EMA(data, period, active):
    closing_prices = _extract_close(data, active)
    return ta.EMA(closing_prices, period)

In [10]:
def RSI(data, period, active):
    closing_prices = _extract_close(data, active)
    return ta.RSI(closing_prices, period)

In [11]:
def ADX(data, period, active):
    closing_prices = _extract_close(data, active)
    high_prices = _extract_high(data, active)
    low_prices = _extract_low(data, active)
    return ta.ADX(high_prices, low_prices, closing_prices, period)

In [12]:
def MACD(data, active, fast_period = 12, slowperiod = 26, signalperiod  = 9):
    closing_prices = _extract_close(data, active)
    return ta.MACD(closing_prices, fastperiod, slowperiod, signalperiod)[0]

In [13]:
def STOCH(data, active):
    closing_prices = _extract_close(data, active)
    high_prices = _extract_high(data, active)
    low_prices = _extract_low(data, active)
    return ta.STOCH(high_prices, low_prices, closing_prices)

### Cálculo de indicadores

In [14]:
for i in set(df.active):
    df.loc[df.active == i, 'EMA5'] = EMA(df, 5, i)
    df.loc[df.active == i, 'EMA15'] = EMA(df, 15, i)
    df.loc[df.active == i, 'EMA50'] = EMA(df, 50, i)
    df.loc[df.active == i, 'EMA100'] = EMA(df, 100, i)
    df.loc[df.active == i, 'EMA200'] = EMA(df, 200, i)
    df.loc[df.active == i, 'RSI5'] = RSI(df, 5, i)
    df.loc[df.active == i, 'RSI15'] = RSI(df, 15, i)
    df.loc[df.active == i, 'RSI50'] = RSI(df, 50, i)
    df.loc[df.active == i, 'RSI100'] = RSI(df, 100, i)
    df.loc[df.active == i, 'RSI200'] = RSI(df, 200, i)
    df.loc[df.active == i, 'ADX5'] = ADX(df, 5, i)
    df.loc[df.active == i, 'ADX15'] = ADX(df, 15, i)
    df.loc[df.active == i, 'ADX50'] = ADX(df, 50, i)
    df.loc[df.active == i, 'ADX100'] = ADX(df, 100, i)
    df.loc[df.active == i, 'ADX200'] = ADX(df, 200, i)
    #df.loc[df.active == i, 'STOCH'] = STOCH(df, i)
    #df.loc[df.active == i, 'MACD'] = MACD(df, i)

df

,open,high,low,close,volume,active,day,month,year,weekday,...,RSI5,RSI15,RSI50,RSI100,RSI200,ADX5,ADX15,ADX50,ADX100,ADX200
0,266.10000,275.00000,262.70000,272.30000,3.702528e+04,AAVEUSDT,3,1,2022,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,312.10000,316.90000,310.20000,312.40000,1.528257e+04,AAVEUSDT,25,10,2021,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,312.40000,312.50000,308.90000,311.60000,7.623416e+03,AAVEUSDT,25,10,2021,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,311.60000,322.50000,311.10000,322.10000,1.825833e+04,AAVEUSDT,25,10,2021,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,321.90000,331.70000,319.60000,330.20000,3.430027e+04,AAVEUSDT,26,10,2021,Tuesday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116024,0.22547,0.22813,0.22489,0.22683,8.140340e+06,XRPUSDT,29,11,2019,Friday,...,43.590048,52.874555,49.536721,49.119075,49.490936,36.771713,11.626407,8.016578,5.430498,2.917235
116025,0.22446,0.22610,0.22356,0.22547,7.884846e+06,XRPUSDT,29,11,2019,Friday,...,37.466221,50.575454,49.044137,48.920026,49.405829,33.407946,11.180673,7.915301,5.411283,2.910862
116026,0.22340,0.22555,0.22245,0.22446,8.512972e+06,XRPUSDT,28,11,2019,Thursday,...,33.143771,48.884138,48.677332,48.771766,49.342497,33.455666,10.476978,7.832204,5.395510,2.905229
116027,0.22036,0.22038,0.21780,0.21867,6.892048e+06,XRPUSDT,3,12,2019,Tuesday,...,18.143893,40.554206,46.636956,47.930617,48.980754,39.764507,10.782038,7.816900,5.393410,2.902579


In [15]:
df['SMA_5'] = df.groupby('active')['close'].transform(lambda x: x.rolling(window = 5).mean())
df['SMA_15'] = df.groupby('active')['close'].transform(lambda x: x.rolling(window = 15).mean())
df['SMA_ratio'] = df['SMA_15'] / df['SMA_5']

In [16]:
df['SMA5_Volume'] = df.groupby('active')['volume'].transform(lambda x: x.rolling(window = 5).mean())
df['SMA15_Volume'] = df.groupby('active')['volume'].transform(lambda x: x.rolling(window = 15).mean())
df['SMA_Volume_Ratio'] = df['SMA5_Volume']/df['SMA15_Volume']

In [17]:
df.head(10)

,open,high,low,close,volume,active,day,month,year,weekday,...,ADX15,ADX50,ADX100,ADX200,SMA_5,SMA_15,SMA_ratio,SMA5_Volume,SMA15_Volume,SMA_Volume_Ratio
0,266.1,275.0,262.7,272.3,37025.276,AAVEUSDT,3,1,2022,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,312.1,316.9,310.2,312.4,15282.570,AAVEUSDT,25,10,2021,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,312.4,312.5,308.9,311.6,7623.416,AAVEUSDT,25,10,2021,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,311.6,322.5,311.1,322.1,18258.332,AAVEUSDT,25,10,2021,Monday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,321.9,331.7,319.6,330.2,34300.271,AAVEUSDT,26,10,2021,Tuesday,...,NaN,NaN,NaN,NaN,309.72,NaN,NaN,22497.9730,NaN,NaN
5,330.1,337.7,326.4,337.6,36722.766,AAVEUSDT,26,10,2021,Tuesday,...,NaN,NaN,NaN,NaN,322.78,NaN,NaN,22437.4710,NaN,NaN
6,337.6,340.4,329.4,335.2,39565.776,AAVEUSDT,26,10,2021,Tuesday,...,NaN,NaN,NaN,NaN,327.34,NaN,NaN,27294.1122,NaN,NaN
7,335.2,344.0,332.3,343.6,35784.010,AAVEUSDT,26,10,2021,Tuesday,...,NaN,NaN,NaN,NaN,333.74,NaN,NaN,32926.2310,NaN,NaN
8,343.6,345.5,325.6,328.8,33407.023,AAVEUSDT,26,10,2021,Tuesday,...,NaN,NaN,NaN,NaN,335.08,NaN,NaN,35955.9692,NaN,NaN
9,328.8,333.6,326.3,332.3,16489.925,AAVEUSDT,26,10,2021,Tuesday,...,NaN,NaN,NaN,NaN,335.50,NaN,NaN,32393.9000,NaN,NaN
